In [1]:
from glob import glob
from random import sample
import json
from Levenshtein import distance as levenshtein_distance
import re

import texte


In [2]:
path = "Corpus/Mazarinades/*/*.xml"

files = glob(path)

print(len(files))

sample_mode = False

if sample_mode:
    files = sample(files, 200)
#     collection_textes = [texte.Texte(file) for file in tqdm(files)]
# else:
collection_textes = [e for e in texte.corpora(files) if e.plain]

3065


 29%|██▉       | 892/3065 [00:09<00:17, 122.25it/s]

Empty file: self.path = 'Corpus/Mazarinades/1201-1300/Moreau1240_GALL.xml'


100%|██████████| 3065/3065 [00:32<00:00, 93.11it/s] 


In [ ]:
with open("LGERM.json", "r") as f:
    lexique = json.load(f)

lexique = set(e.lower() for e in lexique)

chriffre_romain = re.compile("[IVXLCDM]+\.?")

def corrige_mot(mot):
    if chriffre_romain.fullmatch(mot):
        return (mot, "chiffre romain")

    if mot == "&amp;":
        return ("&", "ampersand")

    if mot.lower() in lexique:
        return (mot, "lexique")
    else:
        return (min(lexique, key=lambda x: levenshtein_distance(x, mot.lower())), "levenshtein")

with open("extremes/lexicalite_min.txt", "r") as f:
    hapaxmax = f.read()

for mot in hapaxmax.split():
    correction, type_ = corrige_mot(mot)
    if type_ == "levenshtein":
        print(mot, correction)


GALLICINIVM gallicanvs
NVPER nver
AVDITIVM avditive
FRANCIAM francia
EXPILANTE expirante
MAZARINO mazarine
Gallo galle
intelligentiam intelligentsia
? y
Iob. iob
38. .i.
praeparat preparat
eſcam escap
ſuam. uamp
Ibid. ibid
PARIS, paris
GALLICINIVM gallicanvs
NVPER nver
AVDITIVM avditive
Franciams francivms
expilate expirante
Mazarino. mazarine
etaremRegine efarerent
matriper matricer
ſe¬ ren
licet: licet
Surglre surgelée
prneipsRegpi¬ preparerait
claruit, charuiez
Quemadmodum quemandât
anſer anter
